In [1]:
#We wil pass all the characters in the form of a vector
#We will pass the characters in the form of One Hot Representation

In [47]:
from keras.models import Model
from keras.layers import LSTM,Dense,Activation,Input
import numpy as np

In [3]:
batch_size=64#batch size for traning
epochs=100#Nuber of Epochs to train for
latent_dim=256#Latent Dimensions for the encoding space
num_samples=10000#no. of samples to train on

data_path='fra.txt'

In [4]:
input_texts=[]
target_texts=[]
input_char=set()
target_char=set()

with open(data_path,encoding='utf-8') as f:
    lines=f.read().split('\n')
    for line in lines[:min(num_samples,len(lines) -1)]:
        input_text,target_text,_=line.split('\t')
        #We us tab as the 'start sequence' character
        #for thetargets and '\n' as end sequence character
        
        target_text='\t' + target_text + '\n'
        target_texts.append(target_text)
        input_texts.append(input_text)
        
        for char in input_text:
            if char not in input_char:
                input_char.add(char)
        for char in target_text:
            if char not in target_char:
                target_char.add(char)
                

In [5]:
target_texts

['\tVa !\n',
 '\tSalut !\n',
 '\tSalut.\n',
 '\tCours\u202f!\n',
 '\tCourez\u202f!\n',
 '\tQui ?\n',
 '\tÇa alors\u202f!\n',
 '\tAu feu !\n',
 "\tÀ l'aide\u202f!\n",
 '\tSaute.\n',
 '\tÇa suffit\u202f!\n',
 '\tStop\u202f!\n',
 '\tArrête-toi !\n',
 '\tAttends !\n',
 '\tAttendez !\n',
 '\tPoursuis.\n',
 '\tContinuez.\n',
 '\tPoursuivez.\n',
 '\tBonjour !\n',
 '\tSalut !\n',
 '\tJe comprends.\n',
 "\tJ'essaye.\n",
 "\tJ'ai gagné !\n",
 "\tJe l'ai emporté !\n",
 '\tJ’ai gagné.\n',
 '\tOh non !\n',
 '\tAttaque !\n',
 '\tAttaquez !\n',
 '\tSanté !\n',
 '\tÀ votre santé !\n',
 '\tMerci !\n',
 '\tTchin-tchin !\n',
 '\tLève-toi.\n',
 '\tVa, maintenant.\n',
 '\tAllez-y maintenant.\n',
 '\tVas-y maintenant.\n',
 "\tJ'ai pigé !\n",
 '\tCompris !\n',
 '\tPigé\u202f?\n',
 '\tCompris\u202f?\n',
 "\tT'as capté\u202f?\n",
 '\tMonte.\n',
 '\tMontez.\n',
 '\tSerre-moi dans tes bras !\n',
 '\tSerrez-moi dans vos bras !\n',
 '\tJe suis tombée.\n',
 '\tJe suis tombé.\n',
 "\tJ'ai fui.\n",
 '\tJe sais.\n',
 

In [6]:
len(target_char)

93

In [7]:
print('Number of Samples{}'.format(len(input_texts)))
print('Number of Characters in English {}'.format(len(input_char)))
print('Number of Characters in Fench {}'.format(len(target_char)))
max_encoder_len=max([len(txt) for txt in input_texts])
print(max_encoder_len)
max_decoder_seq_len=max([len(txt) for txt in target_texts])
print(max_decoder_seq_len)

Number of Samples10000
Number of Characters in English 71
Number of Characters in Fench 93
15
59


In [8]:
input_char

{' ',
 '!',
 '$',
 '%',
 '&',
 "'",
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'Y',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 'é'}

In [9]:
sorted_input=sorted(list(input_char))
input_token_index=dict([(char,i) for (i,char) in enumerate(sorted_input)])

In [10]:
input_token_index

{' ': 0,
 '!': 1,
 '$': 2,
 '%': 3,
 '&': 4,
 "'": 5,
 ',': 6,
 '-': 7,
 '.': 8,
 '0': 9,
 '1': 10,
 '2': 11,
 '3': 12,
 '5': 13,
 '6': 14,
 '7': 15,
 '8': 16,
 '9': 17,
 ':': 18,
 '?': 19,
 'A': 20,
 'B': 21,
 'C': 22,
 'D': 23,
 'E': 24,
 'F': 25,
 'G': 26,
 'H': 27,
 'I': 28,
 'J': 29,
 'K': 30,
 'L': 31,
 'M': 32,
 'N': 33,
 'O': 34,
 'P': 35,
 'Q': 36,
 'R': 37,
 'S': 38,
 'T': 39,
 'U': 40,
 'V': 41,
 'W': 42,
 'Y': 43,
 'a': 44,
 'b': 45,
 'c': 46,
 'd': 47,
 'e': 48,
 'f': 49,
 'g': 50,
 'h': 51,
 'i': 52,
 'j': 53,
 'k': 54,
 'l': 55,
 'm': 56,
 'n': 57,
 'o': 58,
 'p': 59,
 'q': 60,
 'r': 61,
 's': 62,
 't': 63,
 'u': 64,
 'v': 65,
 'w': 66,
 'x': 67,
 'y': 68,
 'z': 69,
 'é': 70}

In [11]:
sorted_target=sorted(list(target_char))
target_token_index=dict([(char,i) for (i,char) in enumerate(sorted_target)])

In [12]:
target_token_index

{'\t': 0,
 '\n': 1,
 ' ': 2,
 '!': 3,
 '$': 4,
 '%': 5,
 '&': 6,
 "'": 7,
 '(': 8,
 ')': 9,
 ',': 10,
 '-': 11,
 '.': 12,
 '0': 13,
 '1': 14,
 '2': 15,
 '3': 16,
 '5': 17,
 '8': 18,
 '9': 19,
 ':': 20,
 '?': 21,
 'A': 22,
 'B': 23,
 'C': 24,
 'D': 25,
 'E': 26,
 'F': 27,
 'G': 28,
 'H': 29,
 'I': 30,
 'J': 31,
 'K': 32,
 'L': 33,
 'M': 34,
 'N': 35,
 'O': 36,
 'P': 37,
 'Q': 38,
 'R': 39,
 'S': 40,
 'T': 41,
 'U': 42,
 'V': 43,
 'Y': 44,
 'a': 45,
 'b': 46,
 'c': 47,
 'd': 48,
 'e': 49,
 'f': 50,
 'g': 51,
 'h': 52,
 'i': 53,
 'j': 54,
 'k': 55,
 'l': 56,
 'm': 57,
 'n': 58,
 'o': 59,
 'p': 60,
 'q': 61,
 'r': 62,
 's': 63,
 't': 64,
 'u': 65,
 'v': 66,
 'x': 67,
 'y': 68,
 'z': 69,
 '\xa0': 70,
 '«': 71,
 '»': 72,
 'À': 73,
 'Ç': 74,
 'É': 75,
 'Ê': 76,
 'à': 77,
 'â': 78,
 'ç': 79,
 'è': 80,
 'é': 81,
 'ê': 82,
 'ë': 83,
 'î': 84,
 'ï': 85,
 'ô': 86,
 'ù': 87,
 'û': 88,
 'œ': 89,
 '\u2009': 90,
 '’': 91,
 '\u202f': 92}

In [13]:
#We are One-Hot encoding in the below Step


In [14]:
encoder_input_data=np.zeros(
(len(input_texts),max_encoder_len,len(input_char)),dtype='float32')

In [20]:
encoder_input_data.shape

(10000, 15, 71)

In [16]:
decoder_input_data=np.zeros(
(len(input_texts),max_decoder_seq_len,len(target_char)),
dtype='float64')

In [17]:
decoder_target_data=np.zeros(
(len(input_texts),max_decoder_seq_len,len(target_char)),
dtype='float64')

In [29]:
for i,(input_text,target_text) in enumerate(zip(input_texts,target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i,t,input_token_index[char]]=1.
    encoder_input_data[i,t+1:, input_token_index[' ']]=1.
    
    for t, char in enumerate(target_text):
        decoder_input_data[i,t,target_token_index[char]]=1.
        if t>0:
            decoder_target_data[i,t-1,target_token_index[char]]=1.
    
    decoder_input_data[i,t+1:,target_token_index[char]]=1.
    decoder_target_data[i,t:,target_token_index[' ']]=1.
        

In [36]:
encoder_input_data.shape

(10000, 15, 71)

In [35]:
encoder_input_data[0].shape

(15, 71)

In [37]:
#Define an input sequence and process it
encoder_inputs=Input(shape=(None,len(input_char)))
encoder=LSTM(latent_dim,return_state=True)
encoder_outputs,state_h , state_c=encoder(encoder_inputs)
#We dont require Output so we dont need encoder state variable
encoder_states=[state_h,state_c]

In [38]:
print(encoder_outputs,state_h)

Tensor("lstm_1/strided_slice_18:0", shape=(None, 256), dtype=float32) Tensor("lstm_1/while:4", shape=(None, 256), dtype=float32)


In [40]:
decoder_input=Input(shape=(None,len(target_char)))

In [42]:
decoder_lstm=LSTM(latent_dim,return_sequences=True,return_state=True)
decoder_outputs, _, _=decoder_lstm(decoder_input,initial_state=encoder_states)
decoder_dense=Dense(len(target_char),activation='softmax')
decoder_outputs=decoder_dense(decoder_outputs)

In [48]:
model=Model([encoder_inputs,decoder_input],decoder_outputs)

In [51]:
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])


In [52]:
model.fit([encoder_input_data,decoder_input_data],decoder_target_data,batch_size=batch_size,
         epochs=epochs,
         validation_split=0.2)

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 60s 7ms/step - loss: 1.1721 - accuracy: 0.7246 - val_loss: 1.0529 - val_accuracy: 0.7058
Epoch 2/100
8000/8000 [==============================] - 60s 8ms/step - loss: 0.8191 - accuracy: 0.7738 - val_loss: 0.8014 - val_accuracy: 0.7680
Epoch 3/100
8000/8000 [==============================] - 60s 7ms/step - loss: 0.6503 - accuracy: 0.8149 - val_loss: 0.6907 - val_accuracy: 0.7950
Epoch 4/100
8000/8000 [==============================] - 60s 7ms/step - loss: 0.5750 - accuracy: 0.8330 - val_loss: 0.6419 - val_accuracy: 0.8109
Epoch 5/100
8000/8000 [==============================] - 60s 8ms/step - loss: 0.5278 - accuracy: 0.8462 - val_loss: 0.6072 - val_accuracy: 0.8200
Epoch 6/100
8000/8000 [==============================] - 63s 8ms/step - loss: 0.4931 - accuracy: 0.8551 - val_loss: 0.5686 - val_accuracy: 0.8324
Epoch 7/100
8000/8000 [==============================] - 63s 8ms/step - loss

8000/8000 [==============================] - 68s 9ms/step - loss: 0.0934 - accuracy: 0.9707 - val_loss: 0.5638 - val_accuracy: 0.8754
Epoch 57/100
8000/8000 [==============================] - 66s 8ms/step - loss: 0.0911 - accuracy: 0.9711 - val_loss: 0.5758 - val_accuracy: 0.8740
Epoch 58/100
8000/8000 [==============================] - 67s 8ms/step - loss: 0.0889 - accuracy: 0.9719 - val_loss: 0.5775 - val_accuracy: 0.8741
Epoch 59/100
8000/8000 [==============================] - 71s 9ms/step - loss: 0.0865 - accuracy: 0.9726 - val_loss: 0.5863 - val_accuracy: 0.8746
Epoch 60/100
8000/8000 [==============================] - 78s 10ms/step - loss: 0.0847 - accuracy: 0.9731 - val_loss: 0.5912 - val_accuracy: 0.8738
Epoch 61/100
8000/8000 [==============================] - 73s 9ms/step - loss: 0.0830 - accuracy: 0.9737 - val_loss: 0.5951 - val_accuracy: 0.8738
Epoch 62/100
8000/8000 [==============================] - 75s 9ms/step - loss: 0.0806 - accuracy: 0.9744 - val_loss: 0.5982 - val_

In [57]:
encoder_model=Model(encoder_inputs,encoder_states)

In [58]:
decoder_state_input_h=Input(shape=(latent_dim,))
decoder_state_input_c=Input(shape=(latent_dim,))
decoder_states_inputs=[decoder_state_input_h,decoder_state_input_c]

decoder_outputs,state_h,state_c=decoder_lstm(decoder_input,initial_state=decoder_states_inputs)

decoder_states=[state_h,state_c]
decoder_outputs=decoder_dense(decoder_outputs)

decoder_model=Model([decoder_input]+decoder_states_inputs,
                   [decoder_outputs]+decoder_states)

In [74]:
def decoder_sequence(input_seq):
    #Encode the input as state Vetors
    states_value=encoder_model.predict(input_seq)
    
    #Generate the empty target sequence of length 1
    target_seq=np.zeros((1,1,len(target_char)))
    
    #populate the first character of target sequence with the start character
    target_seq[0,0,target_token_index['\t']]=1.
    
    #sampling loop for a batch of sequences
    #to simplify here we assume batch size of 1
    
    stop_condition=False
    decoded_sentence=''
    while not stop_condition:
        output_tokens,h,c=decoder_model.predict([target_seq]+states_value)
        
        sampled_token_index=np.argmax(output_tokens[0,-1,:])
        sampled_char=reverse_target_char_index[sampled_token_index]
        decoded_sentence+=sampled_char
        
        if(sampled_char=='\n' or len(decoded_sentence)>max_decoder_seq_len):
            stop_condition=True
        
        target_seq=np.zeros((1,1,len(target_char)))
        target_seq[0,0,sampled_token_index]=1.
        
        states_value=[h,c]
        
    return decoded_sentence
    

In [76]:
for seq_index in range(100):
    input_seq=encoder_input_data[seq_index: seq_index+1]
    decoded_sentence=decoder_sequence(input_seq)
    print('-')
    print('input_seq', input_texts[seq_index])
    print('output_seq',decoded_sentence)

-
input_seq Go.
output_seq Va !

-
input_seq Hi.
output_seq Salut.

-
input_seq Hi.
output_seq Salut.

-
input_seq Run!
output_seq Courez !

-
input_seq Run!
output_seq Courez !

-
input_seq Who?
output_seq Qui ?

-
input_seq Wow!
output_seq Ça alors !

-
input_seq Fire!
output_seq Au feu !

-
input_seq Help!
output_seq À l'aide !

-
input_seq Jump.
output_seq Saute.

-
input_seq Stop!
output_seq Stop !

-
input_seq Stop!
output_seq Stop !

-
input_seq Stop!
output_seq Stop !

-
input_seq Wait!
output_seq Attendez !

-
input_seq Wait!
output_seq Attendez !

-
input_seq Go on.
output_seq Continuez.

-
input_seq Go on.
output_seq Continuez.

-
input_seq Go on.
output_seq Continuez.

-
input_seq Hello!
output_seq Salut !

-
input_seq Hello!
output_seq Salut !

-
input_seq I see.
output_seq Je comprends.

-
input_seq I try.
output_seq J'essaye.

-
input_seq I won!
output_seq J'ai gagné !

-
input_seq I won!
output_seq J'ai gagné !

-
input_seq I won.
output_seq J’ai gagné.

-
input_seq Oh 

In [72]:
reverse_input_char_index=dict(
(i,char) for char, i in input_token_index.items())
reverse_target_char_index=dict(
(i,char) for char, i in target_token_index.items())